In [1]:
using BenchmarkTools

In [18]:
function rotate(matrix)
    return reverse(reverse(matrix, dims=1), dims=2)
end

rotate (generic function with 1 method)

In [2]:
function Convolution_2d(input, kernel; bias=0., padding=false)
    input_rows, input_columns = size(input)
    kernel_height, kernel_width = size(kernel)

    if padding
        padded_input = zeros(Float32, input_rows + 2*kernel_height - 2, input_columns + 2*kernel_width - 2)
        padded_input[kernel_height:end-kernel_height+1, kernel_width:end-kernel_width+1] .= input
        input_rows, input_columns = size(padded_input)
        input = padded_input
    end

    output_rows = input_rows - kernel_height + 1
    output_columns = input_columns - kernel_width + 1
    output = zeros(Float32, output_rows, output_columns)


    for c in 1:output_columns
        for r in 1:output_rows
            patch = @view input[r:r+kernel_height-1, c:c+kernel_width-1]
            output[r, c] = sum(patch .* kernel) + bias
        end
    end
    return output
end

Convolution_2d (generic function with 1 method)

In [3]:
function Conv_backward_v1( input, weights, bias, gradient)
    input_height, input_width, input_channels = size(input)
    output_height, output_width, output_channels = size(gradient)
    kernel_height, kernel_width, _, _ = size(weights)
    
    
    grad_input = zeros(Float32, size(input))
    for k in 1:input_channels
        for c in 1:output_channels
            grad_input[:, :, k] += Convolution_2d(weights[:, :, k, c], gradient[:, :, c]; padding=true)
        end
    end

    grad_weights = zeros(Float32, size(weights))
    for k in 1:input_channels
        for c in 1:output_channels
            grad_weights[:, :, k, c] += Convolution_2d(input[:, :, k], gradient[:, :, c])
        end
    end

    grad_bias = reshape(sum(gradient, dims=(1,2,4)), :)
    
    return grad_input, grad_weights, grad_bias
end

Conv_backward_v1 (generic function with 1 method)

In [23]:
function Conv_backward_v2(input, weights, bias, gradient)
    input_height, input_width, input_channels = size(input)
    output_height, output_width, output_channels = size(gradient)
    kernel_height, kernel_width, _, _ = size(weights)
    
    
    grad_input = zeros(Float32, size(input))
    grad_weights = zeros(Float32, size(weights))
    
    tmp_weights = zeros(Float32, kernel_height, kernel_width)
    tmp_input = zeros(Float32, input_height, input_width)
    tmp_gradient = zeros(Float32, output_height, output_width)
    
    for k in 1:input_channels
        for c in 1:output_channels
            tmp_weights .= @views weights[:, :, k, c]
            tmp_gradient .= @views gradient[:, :, c]
            grad_input[:, :, k] .+= Convolution_2d(rotate(tmp_weights), tmp_gradient; padding=true)
        end
    end

    
    for k in 1:input_channels
        for c in 1:output_channels
            tmp_input .= @views input[:, :, k]
            tmp_gradient .= @views gradient[:, :, c]
            grad_weights[:, :, k, c] += Convolution_2d(tmp_input, tmp_gradient)
        end
    end

    grad_bias = reshape(sum(gradient, dims=(1,2,4)), :)
    
    return grad_input, grad_weights, grad_bias
end

Conv_backward_v2 (generic function with 1 method)

In [24]:
function Conv_backward_v3( input, weights, bias, gradient)
    input_height, input_width, input_channels = size(input)
    output_height, output_width, output_channels = size(gradient)
    kernel_height, kernel_width, _, _ = size(weights)
    
    grad_input = zeros(Float32, size(input))
    grad_weights = zeros(Float32, size(weights))
    
    tmp_weights = zeros(Float32, kernel_height, kernel_width)
    tmp_input = zeros(Float32, input_height, input_width)
    tmp_gradient = zeros(Float32, output_height, output_width)
    
    for k in 1:input_channels
        for c in 1:output_channels
            tmp_weights .= @views weights[:, :, k, c]
            tmp_gradient .= @views gradient[:, :, c]
            for i = 1:output_height
                for j = 1:output_width
                    grad_input[i:i+kernel_height-1, j:j+kernel_width-1, k] .+= (weights[:,:,k,c] .* gradient[i,j,c]);
                end
            end
        end
    end
    
    for k in 1:input_channels
        for c in 1:output_channels
            tmp_input .= @views input[:, :, k]
            tmp_gradient .= @views gradient[:, :, c]
            grad_weights[:, :, k, c] += Convolution_2d(tmp_input, tmp_gradient)
        end
    end

    grad_bias = reshape(sum(gradient, dims=(1,2,4)), :)
    
    return grad_input, grad_weights, grad_bias
end

Conv_backward_v3 (generic function with 1 method)

In [25]:
input = rand(Float32, 28, 28, 6);
weights = rand(Float32, 3, 3, 6, 16);
bias = rand(Float32, 16);
gradient = rand(Float32, 26, 26, 16);

In [7]:
@benchmark Conv_backward_v1(input, weights, bias, gradient)

BenchmarkTools.Trial: 72 samples with 1 evaluation.
 Range (min … max):  62.534 ms … 79.064 ms  ┊ GC (min … max): 25.63% … 34.81%
 Time  (median):     71.185 ms              ┊ GC (median):    28.46%
 Time  (mean ± σ):   69.730 ms ±  3.311 ms  ┊ GC (mean ± σ):  28.97% ±  1.24%

                ▄▁ ▁                    ▆█▁                    
  ▄▁▁▁▁▁▁▁▁▁▁▁▆▇██▆█▄▄▄▇▄▁▁▁▁▁▄▁▆▁▁▁▁▁▇▄███▆▆▇▁▄▆▁▄▄▁▁▁▆▁▁▁▄▄ ▁
  62.5 ms         Histogram: frequency by time          76 ms <

 Memory estimate: 216.61 MiB, allocs estimate: 77293.

In [26]:
@benchmark Conv_backward_v2(input, weights, bias, gradient)

BenchmarkTools.Trial: 71 samples with 1 evaluation.
 Range (min … max):  65.261 ms … 79.712 ms  ┊ GC (min … max): 23.82% … 30.71%
 Time  (median):     70.361 ms              ┊ GC (median):    28.87%
 Time  (mean ± σ):   70.592 ms ±  2.068 ms  ┊ GC (mean ± σ):  28.63% ±  1.24%

                           ▁ █▄ ▄                              
  ▃▁▁▁▁▃▅▁▁▁▁▁▃▁▁▁▁▃▁▁▃▁▅▅▆█▅████▆█▅▁▁▃▁▃▅▅▁▃▁▁▁▃▁▁▃▁▃▃▁▁▁▁▁▃ ▁
  65.3 ms         Histogram: frequency by time        75.7 ms <

 Memory estimate: 215.48 MiB, allocs estimate: 77008.

In [27]:
@benchmark Conv_backward_v3(input, weights, bias, gradient)

BenchmarkTools.Trial: 788 samples with 1 evaluation.
 Range (min … max):  6.044 ms …   7.736 ms  ┊ GC (min … max): 0.00% … 17.06%
 Time  (median):     6.244 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   6.351 ms ± 271.312 μs  ┊ GC (mean ± σ):  2.66% ±  3.74%

   ▄▅▁     ▃▇█▄▂                                               
  ▆███▇▅▄▄▅█████▅▄▁▂▂▂▃▄▄▄▅▆▄▃▃▂▂▂▂▃▂▃▃▄▃▄▅▅▄▄▅▄▄▅▃▃▄▂▃▂▁▂▁▂▃ ▃
  6.04 ms         Histogram: frequency by time        7.04 ms <

 Memory estimate: 14.37 MiB, allocs estimate: 131056.

In [28]:
Conv_backward_v2(input, weights, bias, gradient)[1]

28×28×6 Array{Float32, 3}:
[:, :, 1] =
  3.88441   6.96039  11.996   10.509   …  11.4112   9.31517   4.34634
  8.75409  16.8999   26.172   24.351      27.2845  19.1844    8.83958
 12.3549   22.7526   36.951   34.3042     38.4563  25.6959   14.2398
 11.8698   22.3656   35.0585  35.2778     36.274   24.4564   12.4163
 11.224    23.991    34.2808  33.7373     39.5826  24.3518   11.8866
 11.7744   23.3214   33.2305  33.5063  …  40.7506  25.8772   13.4633
 12.2079   22.7542   35.3802  35.3254     37.33    23.8247   13.0403
 12.0157   23.327    35.8079  34.6388     34.0463  24.6082   12.006
 12.0158   23.475    36.5199  37.0609     35.8374  20.2246   10.7585
 13.3238   23.2997   35.6738  36.8049     33.0391  22.2442   11.7562
 12.2097   24.3773   36.6749  35.9608  …  34.7383  20.4461   11.984
 11.8032   24.0198   34.6787  37.4844     34.2321  21.9342   11.1595
 11.7719   22.2831   35.0289  38.4416     37.2384  24.9176   13.5699
  ⋮                                    ⋱   ⋮                 
 1

In [29]:
Conv_backward_v3(input, weights, bias, gradient)[1]

28×28×6 Array{Float32, 3}:
[:, :, 1] =
  4.26964   8.47371  10.9658  10.0447  …  10.8847   5.36437   2.01673
  7.51263  14.7062   24.4414  23.8713     23.2567  15.0588    6.8504
 13.5933   24.3304   35.965   34.8217     34.5327  22.6916   11.1953
 14.1003   23.9915   37.2316  33.4365     36.5308  23.4382   12.2078
 12.3335   23.8053   35.6011  34.9597     34.4145  24.4418   11.522
 11.5787   22.0559   34.2692  36.9034  …  34.3511  22.2462   11.7798
 10.5947   23.5975   36.4918  35.2845     30.547   18.4238   11.3576
 11.26     22.6469   36.1704  38.0293     32.7226  22.1888   11.8926
 11.9897   23.0721   38.4429  37.9149     33.3718  24.0063   11.8758
 11.6464   23.9262   37.8694  37.923      35.0054  23.3621   12.6299
 12.1537   22.3839   35.1913  34.4086  …  32.4013  22.7547   11.8849
 12.4859   24.1915   34.7664  35.3457     34.0624  24.4413   11.8236
 13.0393   23.3916   36.6178  35.563      35.5464  22.0955   12.2327
  ⋮                                    ⋱   ⋮                 
 1